In [1]:
from typing import Dict, List, Tuple, Union, Any, Optional

import os
import sys
import time
print("Python version", sys.version)

# Ignore pandas warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pathlib import Path

import pandas as pd
import numpy as np
import tensorflow as tf
import datetime as dt

from tqdm.autonotebook import tqdm

import ray
from ray import train, tune

from src.datasets import daocensus_text

%load_ext autoreload
%autoreload 2

tf.get_logger().setLevel('WARNING')

tf.config.list_physical_devices("GPU")
sys_details = tf.sysconfig.get_build_info()
cuda = sys_details.get("cuda_version", -1)
cudnn = sys_details.get("cudnn_version", -1)
print(cuda, cudnn)
print('Ray version:', ray.__version__)

Python version 3.9.18 (main, Oct 24 2023, 09:18:18) 
[GCC 11.4.0]


2024-03-22 00:52:59.509098: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 00:52:59.509119: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 00:52:59.509136: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


11.8 8
Ray version: 2.9.1


In [2]:
# Others config
SEED: int = 57
RAY_RESULTS_PATH: Path = Path('~/ray_results').expanduser()

# Dataset config
ORG_NAME = 'Decentraland'
ORG_PLATFORM = 'snapshot'
USE_ORG_NAMES: bool = False
CUTOFF_DATE_STR: str = None

# Dataset splits config
SPLITS_FREQ = 'W-THU' # Split weekly
LAST_SPLITS = 10 # Use just last 10 splits
SPLITS_NORMALIZE = True

# Training config
MAX_EPOCHS: int = 200
EPOCHS_PER_ITER: int = 5
SAMPLES_PER_SPLIT: int = 100
OPTIM_METRIC: str = 'map@10'

# Eval config
TOP_K: List[int] = [5, 10]
METRICS: List[str] = ["recall", "ndcg", "precision", "map"]

In [3]:
# Parameters
ORG_NAME = "MetaCartel - MetaCartel Ventures"
ORG_PLATFORM = ["daohaus", "snapshot"]
SPLITS_FREQ = "W-THU"
SPLITS_NORMALIZE = True
LAST_SPLITS = 10
USE_ORG_NAMES = True
EXECUTION_ID = "2024-03-21"
CUTOFF_DATE_STR = "2020-11-06"


In [4]:
CUTOFF_DATE = dt.datetime.fromisoformat(CUTOFF_DATE_STR) if CUTOFF_DATE_STR else None

# Load data

In [5]:
# dfptext = pd.read_csv('./snapshot_proposals.csv')[['proposal_id', 'title', 'description', 'start', 'end']]
# dfv, dfp = daocensus.get("./data/daos-census", ORG_NAME, 'snapshot')
# dfv['voter'] = dfv['voter'].astype('str')
# dfp = dfp.merge(dfptext, how='left', left_on='platform_proposal', right_on='proposal_id')
# dfp[['start', 'end']] = dfp[['start', 'end']].astype('datetime64')

dfv, dfp = daocensus_text.get("./data/daos-census-text", ORG_NAME, ORG_PLATFORM, use_org_names=USE_ORG_NAMES, cutoff_date=CUTOFF_DATE)
print(dfv.info())
print(dfp.info())

Warning, removing 1105 votes without proposal
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1105 entries, 0 to 1104
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   platform       1105 non-null   object        
 1   name           1105 non-null   object        
 2   id             1105 non-null   object        
 3   proposal       1105 non-null   category      
 4   deployment     1105 non-null   object        
 5   platform_vote  1105 non-null   object        
 6   voter          1105 non-null   category      
 7   date           1105 non-null   datetime64[ns]
 8   choice         1105 non-null   object        
 9   weight         686 non-null    float64       
dtypes: category(2), datetime64[ns](1), float64(1), object(6)
memory usage: 84.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 373 entries, 0 to 372
Data columns (total 13 columns):
 #   Column               Non-Null Count

## Transform data

In [6]:
def to_microsoft(dfv):
    df = dfv[['voter', 'proposal', 'date']].rename(columns={
        'voter': 'userID',
        'proposal': 'itemID',
        'date': 'timestamp',
    })
    df['itemID'] = df['itemID'].astype('str')
    df['rating'] = 1
    return df

df = to_microsoft(dfv)
df.head()

,userID,itemID,timestamp,rating
0,0x0eabffd8ce94ab2387fc44ba32642af0c58af433,b23fab43-3b08-5638-b13d-0e286cadb04b,2020-04-01 03:11:46,1
1,0x0eabffd8ce94ab2387fc44ba32642af0c58af433,18f8c9fd-9796-5802-ab78-b449c65ffb50,2020-06-17 03:08:43,1
2,0x0eabffd8ce94ab2387fc44ba32642af0c58af433,2a062f74-c7cb-5625-b8ba-537b23ae5318,2019-07-25 20:06:53,1
3,0x0eabffd8ce94ab2387fc44ba32642af0c58af433,3eff52ec-93aa-5c41-9ff2-3a18d278d138,2019-07-30 18:13:46,1
4,0x0eabffd8ce94ab2387fc44ba32642af0c58af433,b21a24a0-1a16-55a1-8c0e-c97ed7cf8e69,2019-08-06 18:25:25,1


# Split data

Each proposal remains open for a few days, our environment is different of a movies recommender system. For this reason, we will use a TimeSeriesSplit instead of a K-Fold to cross-validate the model.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_013.png)

El TimeSeriesSplit de scikit-learn no nos vale porque el número de elementos en cada split es el mismo, pero el tamaño del intervalo, no. Como queremos simular un comportamiento realista, haremos el split dividiendo por intervalos de igual longitud.

In [7]:
from recommenders.evaluation.python_evaluation import metrics as metrics_dict

In [8]:
from src.model_selection import timeFreqSplitCurrent, timeIntervalSplitCurrent

# max_train_prev = df['timestamp'].min().date()
# N_SPLITS = 10; SKIP_SPLIT = 0
# folds = list(timeIntervalSplitCurrent(df, N_SPLITS, dfp, skip=SKIP_SPLIT, remove_not_in_train_col='userID', return_open=True))
folds = list(timeFreqSplitCurrent(df, SPLITS_FREQ, dfp, return_open=True, remove_not_in_train_col='userID'))[-LAST_SPLITS:]
for i, (dftrain, dftest, t, open_proposals) in enumerate(folds):
    min_train = dftrain['timestamp'].min().date()
    max_train = dftrain['timestamp'].max().date()
    min_test  = dftest['timestamp'].min().date()
    max_test  = dftest['timestamp'].max().date()

    train_users = len(set(dftrain['userID']))
    test_users = len(set(dftest['userID']))
    
    print(f"Split {i}, train from: {min_train} to {max_train}, test from: {min_test} to {max_test}")
    print(f"  t: {t}")
    print(f"  len(train): {len(dftrain)}, len(test): {len(dftest)}")
    print(f"  users(train): {train_users}, users(test): {test_users}")

    print()
    dftest['prediction'] = 1
    for m in METRICS:
        f = metrics_dict[f'{m}_at_k']
        print(f"  highest possible {m}@{TOP_K[0]}:\t{f(dftest, dftest, k=TOP_K[0], relevancy_method='top_k'):.4f}")

    print("-"*30)

Split 0, train from: 2019-06-06 to 2020-08-19, test from: 2020-08-20 to 2020-08-30
  t: 2020-08-20 00:00:00
  len(train): 764, len(test): 9
  users(train): 76, users(test): 5

  highest possible recall@5:	1.0000
  highest possible ndcg@5:	1.0000
  highest possible precision@5:	0.3600
  highest possible map@5:	1.0000
------------------------------
Split 1, train from: 2019-06-06 to 2020-08-26, test from: 2020-08-28 to 2020-10-07
  t: 2020-08-27 00:00:00
  len(train): 793, len(test): 12
  users(train): 77, users(test): 7

  highest possible recall@5:	1.0000
  highest possible ndcg@5:	1.0000
  highest possible precision@5:	0.3429
  highest possible map@5:	1.0000
------------------------------
Split 2, train from: 2019-06-06 to 2020-09-02, test from: 2020-09-03 to 2020-10-07
  t: 2020-09-03 00:00:00
  len(train): 816, len(test): 16
  users(train): 78, users(test): 10

  highest possible recall@5:	1.0000
  highest possible ndcg@5:	1.0000
  highest possible precision@5:	0.3200
  highest poss

  highest possible precision@5:	0.2667
  highest possible map@5:	1.0000
------------------------------
Split 8, train from: 2019-06-06 to 2020-10-14, test from: 2020-10-15 to 2020-10-23
  t: 2020-10-15 00:00:00
  len(train): 1003, len(test): 13
  users(train): 93, users(test): 12

  highest possible recall@5:	1.0000
  highest possible ndcg@5:	1.0000
  highest possible precision@5:	0.2167
  highest possible map@5:	1.0000
------------------------------
Split 9, train from: 2019-06-06 to 2020-10-21, test from: 2020-10-22 to 2020-10-23
  t: 2020-10-22 00:00:00
  len(train): 1050, len(test): 7
  users(train): 96, users(test): 5

  highest possible recall@5:	1.0000
  highest possible ndcg@5:	1.0000
  highest possible precision@5:	0.2800
  highest possible map@5:	1.0000
------------------------------


# Defining training

In [9]:
# from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from src.models import LightGCNCustom
from recommenders.utils.python_utils import get_top_k_scored_items

## Small test of `LightGCNCustom`

In [10]:
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

In [11]:
hparams = prepare_hparams(
    model_type='lightgcn',
    n_layers=3,
    batch_size=512,
    embed_size=64,
    epochs=2,
    learning_rate=0.001,
    decay=0.001,
    metrics=["recall", "ndcg", "precision", "map"],
    eval_epoch=2,
    top_k=TOP_K[0],
    save_model=False,
    MODEL_DIR='./data/model/lightgcn/',
)
dataloader = ImplicitCF(train=folds[-1][0], test=folds[-1][1], seed=SEED)
print("items:", dataloader.n_items, "user:", dataloader.n_users)
model = LightGCNCustom(data=dataloader, hparams=hparams)

items: 363 user: 96
Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [12]:
model.fit()

Epoch 1 (train)0.1s: train loss = 0.68010 = (mf)0.67940 + (embed)0.00070


Epoch 2 (train)0.0s + (eval)0.5s: train loss = 0.67748 = (mf)0.67677 + (embed)0.00071, recall = 0.06667, ndcg = 0.05922, precision = 0.04000, map = 0.03333


In [13]:
model.run_eval()

[0.06666666666666667, 0.05921638219317306, 0.04, 0.03333333333333333]

In [14]:
model.recommend_k_items(
    dataloader.test, 
    top_k=3, 
    use_id=True, 
    remove_seen=True, 
    recommend_from=folds[-1][3]
)

,userID,itemID,prediction
0,51,173,0.019425
1,51,362,0.011043
2,51,172,0.010538
3,82,173,0.016291
4,82,362,0.013017
5,82,172,0.010675
6,85,362,0.011251
7,85,152,0.006287
8,85,172,0.003683
9,87,151,-0.001944


In [15]:
# problem_prop = 'b926a2eb-e2c7-5d0d-bbec-015efe30edde'

# tr, te, t, open_proposals = folds[-1]
# print("t:", t)
# print("In open?: ", problem_prop in set(open_proposals))
# print("In Train?:", any(tr['itemID'] == problem_prop))
# print("In Test?: ", any(te['itemID'] == problem_prop))
# print("In dfv?:  ", in_dfv := any(dfv['proposal'] == problem_prop))
# if in_dfv:
#     display(dfv[dfv['proposal'] == problem_prop])

# dfp.set_index('id').loc[problem_prop]

In [16]:
# Free the resources
del model
del dataloader

# Defining trainer

In [17]:
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

class TrainLightGCN(tune.Trainable):
    def setup(
        self,
        config: Dict[str, Any],
        data,
    ):
        self.config = config

        self.hparams = prepare_hparams(
            model_type='lightgcn',
            n_layers=config['conv_layers'],
            batch_size=2**config['batch_size'],
            embed_size=config['embedding_dim'],
            epochs=EPOCHS_PER_ITER,
            learning_rate=config['learning_rate'],
            decay=config['l2'],
            metrics=METRICS,
            eval_epoch=-1,
            top_k=TOP_K[0],
            save_model=False,
            MODEL_DIR='./data/model/lightgcn/',
        )

        train, test, self.t, self.open_proposals = data
        self.dataloader = ImplicitCF(train=train, test=test, seed=SEED)
        self.model = LightGCNCustom(self.hparams, self.dataloader, seed=SEED)
        self.total_train = 0
        self.total_eval = 0

    @property
    def iteration(self):
        return self.model.epochs_done

    @property
    def training_iteration(self):
        return self.model.epochs_done

    def step(self):
        """
        As a rule of thumb, the execution time of step should be large enough to avoid overheads 
        (i.e. more than a few seconds), but short enough to report progress periodically 
        (i.e. at most a few minutes).
        """
        assert EPOCHS_PER_ITER > 0

        train_start = time.time()
        for _ in range(EPOCHS_PER_ITER):
            ret = self.model.fit_epoch()
        eval_start = train_end = time.time()

        eval_dict = {'model_'+k:v for k,v in zip(self.model.metrics, self.model.run_eval())}
        for k in TOP_K:
            recs = self.model.recommend_k_items(
                self.dataloader.test, 
                top_k=k,
                use_id=True, 
                remove_seen=True, 
                recommend_from=self.open_proposals,
            )
            
            eval_dict[f'precision@{k}'] = precision_at_k(self.dataloader.test, recs, k=k)
            eval_dict[f'ndcg@{k}'] = ndcg_at_k(self.dataloader.test, recs, k=k)
            eval_dict[f'recall@{k}'] = recall_at_k(self.dataloader.test, recs, k=k)
            eval_dict[f'map@{k}'] = map_at_k(self.dataloader.test, recs, k=k)

        eval_end = time.time()

        self.total_train += train_end - train_start
        self.total_eval += eval_end - eval_start
        
        return {
            'iteration': self.iteration,
            'loss': ret[0],
            'mf_loss': ret[1],
            'emb_loss': ret[2],
            **eval_dict,
            'time_train': train_end-train_start,
            'time_test': eval_end-eval_start,
            'time_total_train': self.total_train,
            'time_total_test': self.total_eval,
        }

    def save_checkpoint(self, checkpoint_dir):
        checkpoint_path = os.path.join(checkpoint_dir, "model")
        self.model.saver.save(
            sess=self.model.sess,
            save_path=checkpoint_path,
        )
        return checkpoint_dir

    def load_checkpoint(self, checkpoint_path):
        self.model.load(checkpoint_path)

# Big experiment

In [18]:
RAY_RESULTS_PATH

PosixPath('/home/daviddavo/ray_results')

In [19]:
MAX_EMBEDDING_DIM = 1024
MAX_BATCH_SIZE = 10 # 2**10
MIN_LR = 1e-4
GPUS = 16

if ORG_NAME in ['PancakeSwap', 'Aave - Aavegotchi']:
    MAX_EMBEDDING_DIM = 256
    MAX_BATCH_SIZE = 14
    GPUS = 4
    MIN_LR = 1e-2
elif ORG_NAME in ['DEAD FoundationsDAO']:
    GPUS = 20

In [20]:
print(os.uname().nodename)

### SET TRAINING RESOURCES
if os.uname().nodename == 'lamarck':
    # assert torch.cuda.is_available()

    NUM_SAMPLES = SAMPLES_PER_SPLIT
    # Every run takes approx half a gig of vram (no optimizations)
    # The RTX 4090 has 24GB so we can run the model about 48 times
    resources_per_trial={
        'cpu': 1,
        'gpu': 1 / GPUS,
    }
else:
    NUM_SAMPLES = 1
    resources_per_trial={
        'cpu': 1,
        # It takes about 1.5 GiB with full training data, but I put a bit more because
        # this notebook also takes a bit of memory
        'memory': 2e9,
    }
print(resources_per_trial)

lamarck
{'cpu': 1, 'gpu': 0.0625}


In [21]:
from ray.tune.search.hyperopt import HyperOptSearch
import datetime as dt
from src import paths

def getTunerOnFold(f, points_to_evaluate = None):
    name = f'LightGCN_optim={OPTIM_METRIC},dao={ORG_NAME},freq={SPLITS_FREQ},normalize={SPLITS_NORMALIZE},cutoff_date={CUTOFF_DATE.isoformat()},fold={f}'
    paths = list(RAY_RESULTS_PATH.glob(f'{name}_*'))
    last_experiment = max(paths, key=lambda x: x.stat().st_ctime) if paths else None

    dftrain,dftest,t,open_proposals = folds[f]
    param_space = dict(
        fold=f,
        batch_size=tune.randint(6, MAX_BATCH_SIZE), # 64 - 1024
        embedding_dim=tune.lograndint(1, MAX_EMBEDDING_DIM, base=2),
        conv_layers=tune.randint(1,6),
        learning_rate=tune.qloguniform(MIN_LR, 1, 1e-4),
        l2=tune.loguniform(1e-7, 1e-2, 1e-7),
    )
    
    ### RESTORE EXPERIMENT OR CREATE A NEW ONE
    if last_experiment and tune.Tuner.can_restore(last_experiment):
        print(f"Restoring last experiment: {last_experiment}")
        tuner = tune.Tuner.restore(
            str(last_experiment),
            trainable=tune.with_resources(
                # tune.with_parameters(TrainLightGCN,  train=dftrain, test=dftest, open_proposals=open_proposals),
                tune.with_parameters(TrainLightGCN, data=folds[f]),
                resources_per_trial,
            ),
            restart_errored=True,
            param_space=param_space,
        )
    else:
        print(f"No experiment found for fold {f}, creating new tuner with {NUM_SAMPLES} samples")
        search_alg = None
        search_alg = HyperOptSearch(
            # points_to_evaluate=[{
            #     'batch_size': 8, # 2**8 = 256
            #     'learning_rate': 10e-2,
            #     'l2': 10e-6,
            #     'embedding_dim': 100,
            #     'conv_layers': 3,
            # }],
            points_to_evaluate = points_to_evaluate,
            random_state_seed=SEED,
        )
        # search_alg = tune.search.Repeater(search_alg, N_SPLITS-SKIP_SPLIT)
        
        tuner = tune.Tuner(
            tune.with_resources(
                # tune.with_parameters(TrainLightGCN,  train=dftrain, test=dftest, open_proposals=open_proposals),
                tune.with_parameters(TrainLightGCN, data=folds[f]),
                resources_per_trial,
            ),
            run_config=train.RunConfig(
                stop={'training_iteration': MAX_EPOCHS/EPOCHS_PER_ITER, 'time_total_train': 300},
                name=name + f'_{dt.datetime.now().isoformat()}',
                storage_path=RAY_RESULTS_PATH,
                # failure_config=train.FailureConfig(fail_fast='raise'),
                failure_config=train.FailureConfig(max_failures=3),
            ),
            param_space=param_space,
            tune_config=tune.TuneConfig(
                search_alg=search_alg,
                num_samples=NUM_SAMPLES,
                metric=OPTIM_METRIC,
                mode='max',
            )
        )

    return tuner

In [22]:
import logging

def findConfig(rg):
    for r in rg:
        if r.config:
            msk = [r.config[k] == v for k, v in last_best_result.config.items() if k != 'fold']
            if all(msk):
                return r
            elif sum(msk) >= 3:
                print(r.config, msk)

    return None

tuners = []
results = []
last_best_result = None
for f in range(LAST_SPLITS):
    best_prev_config = None
    if last_best_result is not None:
        best_prev_config = last_best_result.config.copy()
        best_prev_config['fold'] += 1
        best_prev_config = [best_prev_config]
    
    t = getTunerOnFold(f, best_prev_config)
    tuners.append(t)

    rg = t.fit()
    assert rg.num_errors == 0, f"There are {rg.num_errors} errors"
    assert rg.num_terminated == NUM_SAMPLES, f'Some samples are not terminated ({rg.num_terminated} != {NUM_SAMPLES})'
    results.append(rg)

    # Assert that the prev config has been tried
    if last_best_result is not None:
        # if not any( 
        #     all((r.config[k] == v for k, v in last_best_result.config.items() if k != 'fold'))
        #     for r in rg if r.config
        # ):
        if not findConfig(rg):
            print("Best config:", last_best_result.config)
            assert False, f"The best config from previous fold has not been tested in fold {f}"
        else:
            logging.info(f'Fold {f}. Best prev result was {last_best_result.path} and config has been found {findConfig(rg).path}')
    
    last_best_result = rg.get_best_result()

(pid=3880727) 2024-03-22 01:47:50.007206: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=3880727) 2024-03-22 01:47:50.007234: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=3880727) 2024-03-22 01:47:50.007248: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(TrainLightGCN pid=3880727) /home/daviddavo/upm-tfm-notebooks/.direnv/python-3.9/lib/python3.9/site-packages/recommenders/models/deeprec/DataModel/ImplicitCF.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(TrainLightGCN pid=3880727)   df = train if test is None else train.append(test)


(TrainLightGCN pid=3880727) Already create adjacency matrix. [repeated 2x across cluster]
(TrainLightGCN pid=3880727) Already normalize adjacency matrix. [repeated 2x across cluster]
(TrainLightGCN pid=3880515) Using xavier initialization.
(TrainLightGCN pid=3880727) Using xavier initialization.


(pid=3880862) 2024-03-22 01:47:53.552065: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=3880862) 2024-03-22 01:47:53.552099: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=3880862) 2024-03-22 01:47:53.552112: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(TrainLightGCN pid=3880727) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/daviddavo/ray_results/LightGCN_optim=map@10,dao=MetaCartel - MetaCartel Ventures,freq=W-THU,normalize=True,cutoff_date=2020-11-06T00:00:00,fold=9_2024-03-22T01:47:48.988892/TrainLightGCN_198c29f6_1_batch_size=7,conv_layers=2,embedding_dim=1,fold=9,l2=0.0001,learning_rate=0.0006_2024-03-22_01-47-48/checkpoint_000000)


...

(TrainLightGCN pid=3897070) /home/daviddavo/upm-tfm-notebooks/.direnv/python-3.9/lib/python3.9/site-packages/recommenders/models/deeprec/DataModel/ImplicitCF.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(TrainLightGCN pid=3897070)   df = train if test is None else train.append(test)


(TrainLightGCN pid=3896897) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/daviddavo/ray_results/LightGCN_optim=map@10,dao=MetaCartel - MetaCartel Ventures,freq=W-THU,normalize=True,cutoff_date=2020-11-06T00:00:00,fold=9_2024-03-22T01:47:48.988892/TrainLightGCN_817cfe74_98_batch_size=8,conv_layers=5,embedding_dim=463,fold=9,l2=0.0000,learning_rate=0.0861_2024-03-22_01-53-40/checkpoint_000000)


(TrainLightGCN pid=3897070) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/daviddavo/ray_results/LightGCN_optim=map@10,dao=MetaCartel - MetaCartel Ventures,freq=W-THU,normalize=True,cutoff_date=2020-11-06T00:00:00,fold=9_2024-03-22T01:47:48.988892/TrainLightGCN_b7ce5cac_99_batch_size=9,conv_layers=1,embedding_dim=12,fold=9,l2=0.0018,learning_rate=0.0066_2024-03-22_01-53-44/checkpoint_000000)
(pid=3897236) 2024-03-22 01:53:49.206023: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=3897236) 2024-03-22 01:53:49.206051: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=3897236) 2024-03-22 01:53:49.206064: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to registe

(TrainLightGCN pid=3897236) /home/daviddavo/upm-tfm-notebooks/.direnv/python-3.9/lib/python3.9/site-packages/recommenders/models/deeprec/DataModel/ImplicitCF.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(TrainLightGCN pid=3897236)   df = train if test is None else train.append(test)


2024-03-22 01:53:54,762	INFO tune.py:1042 -- Total run time: 365.77 seconds (365.74 seconds for the tuning loop).


In [23]:
# This is needed for papermill to run the whole notebook and not stop above
# because ray tune catches the exception

print("All finished!")

All finished!
